In [1]:
import string
import json
import random
import nltk
import datetime
import numpy as np
import tensorflow as tf
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from difflib import SequenceMatcher
from tensorflow.keras.utils import plot_model


# Download NLTK data (run these once)
nltk.download("punkt")
nltk.download("wordnet")

# Load JSON data
with open(r"C:\Users\fsafi\OneDrive\intents.json", "r", encoding="utf-8") as json_file:
    data = json.load(json_file)
# Initialize data structures
lemmatizer = WordNetLemmatizer()
words = []
classes = []
documents = []
responses = []

# Process data
for intent in data['intents']:
    for pattern in intent['patterns']:
        tokens = nltk.word_tokenize(pattern)
        words.extend(tokens)
        documents.append((tokens, intent['tag']))
        responses.extend(intent['responses'])
    if intent['tag'] not in classes:
        classes.append(intent['tag'])

# Preprocess words and classes
words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in string.punctuation]
words = sorted(set(words))
classes = sorted(set(classes))

# Prepare training data
training = []
out_empty = [0] * len(classes)

for doc, tag in documents:
    bag = [1 if word in doc else 0 for word in words]
    output_row = list(out_empty)
    output_row[classes.index(tag)] = 1
    training.append([bag, output_row])

random.shuffle(training)
train_x = np.array([bag for bag, _ in training])
train_y = np.array([output_row for _, output_row in training])

# Define model architecture
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Define learning rate schedule
initial_learning_rate = 0.01
lr_schedule = ExponentialDecay(
    initial_learning_rate, decay_steps=10000, decay_rate=0.9
)

# Compile the model
model.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
    metrics=['accuracy']
)

# Train the model
model.fit(x=train_x, y=train_y, epochs=150, verbose=1)

# preprocessing functions and get_response function

def clean_text(text):
    tokens = nltk.word_tokenize(text)
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return tokens

def bag_of_words(text, vocab):
    tokens = clean_text(text)
    bow = [1 if word in tokens else 0 for word in vocab]
    return np.array(bow)

def is_response_similar(response, user_input):
    similarity_threshold = 0.7  # Could be adjusted as needed
    similarity = SequenceMatcher(None, response, user_input).ratio()
    return similarity > similarity_threshold

def pred_class(user_input, vocab, labels):
    bow = bag_of_words(user_input, vocab)
    result = model.predict(np.array([bow]))[0]
    
    # Define the threshold
    threshold = 0.5  # Adjust as needed
    
    # Predicted labels based on the threshold
    predicted_labels = [labels[i] for i, value in enumerate(result) if value > threshold]
    
    # Check if any previous response is similar to the current input
    for response_tuple in conversation_history[::-1]:
        response = response_tuple[0]
        if is_response_similar(response, user_input):
            predicted_labels.append(response)
    
    return predicted_labels


def get_response(intents, data):
    for intent in data['intents']:
        if intent['tag'] in intents:
            return random.choice(intent['responses'])
    return "I'm sorry, I didn't understand that."



# interaction loop
print("Press 0 if you'd like to stop chatting with our ChatBot")

conversation_history = []  # List to store conversation history

while True:
    user_input = input("You: ")
    if user_input == "0":
        break
    
    conversation_history.append(("user", user_input))  # Add user input to history
    
    intents = pred_class(user_input, words, classes)
    response = get_response(intents, data)
    
    conversation_history.append(("chatbot", response))  # Add chatbot response to history
    
    print("ChatBot:", response)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\fsafi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\fsafi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Epoch 1/150
3/3 [==============================] - 1s 6ms/step - loss: 1.6322 - accuracy: 0.1818
Epoch 2/150
3/3 [==============================] - 0s 5ms/step - loss: 1.4210 - accuracy: 0.5000
Epoch 3/150
3/3 [==============================] - 0s 5ms/step - loss: 1.2404 - accuracy: 0.6061
Epoch 4/150
3/3 [==============================] - 0s 5ms/step - loss: 1.1179 - accuracy: 0.6515
Epoch 5/150
3/3 [==============================] - 0s 5ms/step - loss: 0.9282 - accuracy: 0.6818
Epoch 6/150
3/3 [==============================] - 0s 5ms/step - loss: 0.7151 - accuracy: 0.8030
Epoch 7/150
3/3 [==============================] - 0s 5ms/step - loss: 0.6147 - accuracy: 0.8939
Epoch 8/150
3/3 [==============================] - 0s 5ms/step - loss: 0.3598 - accuracy: 0.9848
Epoch 9/150
3/3 [==============================] - 0s 7ms/step - loss: 0.2696 - accuracy: 0.9545
Epoch 10/150
3/3 [==============================] - 0s 5ms/step - loss: 0.2178 - accuracy: 0.9697
Epoch 11/150
3/3 [===========

You:  hello 


1/1 [==============================] - 0s 115ms/step
ChatBot: Yo !


You:  i have wifi problem


...

You:  reinitialization


1/1 [==============================] - 0s 29ms/step
ChatBot: Salut à tous ! Si vous avez des demandes, je suis là pour vous aider.


You:  reset 


1/1 [==============================] - 0s 19ms/step
ChatBot: Salut à toi !


You:  0
